We will finetune a pretrained `MaskRCNN` model in the <b>Penn-Fudan Database for Pedestrian Detection and Segmentation</b>. It contains 170 images with 345 instances of pedestrians, and use many features to train an `instance segmentation` model on a custom dataset.

### Defining Dataset
The reference scripts for training object detection, instance segmentation and person keypoint detection allows for easily supporting adding new `custom datasets`.

The Dataset should inherit from the standard `torch.utils.data.Dataset` class and implement `__len__` and `__getitem__`.
The only specificity that we require is that the dataset `__getitem__` should return:
   1. image: a PIL image of size `(H, W)`.
    
   2. target: a dict containing the following fields
     - `boxes (FloatTensor[N, 4])`: The coordinates of the `N` bounding boxes in `[x0, y0, x1, y1]` format.
     - `labels (Int64Tensor[N])`: The label for each bounding box. `0` represents always the background class.
     - `image_id (Int64Tensor[1])`: an image identifier. It should be unique between all the images in the dataset, and is used during evaluation.
     - `area (Tensor[N])`: The area of the bounding box. This is used during evaluation with the `COCO` metric, to seperate metric scores between small, medium and large boxes.
     - `iscrowd (UInt8Tensor[N])`: instances with iscrowd=True will be ignored during evaluation.
     - (optionally) `masks (UInt8Tensor[N, H, W])`: The segmentation masks for each one of the objects
     - (optionally) `keypoints (FloatTensor[N, K, 3])`: For each one of the N objects, it contains the K keypoints in `[x, y, visibility]` format, defining the object. visibility=0 means that the keypoint is not visible.
     
     
One note on the `labels`. The model considers class `0` as background. If your dataset does not contain the background class, you should not have `0` in your `labels`. For example, assuming you have just two classes, <i>cat</i> and <i>dog</i>, you can have define `1` to represent <i>cats</i> and `2` represent <i>dogs</i>. So, for instance, if one of the images has both classes, your `labels` tensor should look like `[1,2]`.

Additionally, if you want to use aspect ratio grouping during training (so that each batch only contains images with similar aspect ratios), then it is recommended to also implement a `get_height_and_width` method, which returns the height and the width of the image. If this method is not provided, we query all elements of the dataset via `__getitem__` , which loads the image in memory and is slower than if a custom method is provided.

### Write Custom Dataset for PennFudan

<img src="https://pytorch.org/tutorials/_static/img/tv_tutorial/tv_image01.png"/>
<img src="https://pytorch.org/tutorials/_static/img/tv_tutorial/tv_image02.png"/>

So each image has a corresponding segmentation mask, where each color correspond to a different instance. Write `torch.utils.data.Dataset` class for this.

In [1]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset
from PIL import Image

class PennFudanDataset(Dataset):
    def __init__(self, root, transforms):
        self.root = root
        self.transforms = transforms
        # Load all image files, sorting them to ensure
        # that they are aligned
        self.imgs = list(sorted(os.listdir(os.path.join(root, "PNGImages"))))
        self.masks = list(sorted(os.listdir(os.path.join(root, "PedMasks"))))
        
    def __getitem__(self, idx):
        # Load images and masks
        img_path = os.path.join(self.root, "PNGImages", self.imgs[idx])
        mask_path = os.path.join(self.root, "PedMasks", self.masks[idx])
        img = Image.open(img_path).convert("RGB")
        # NOTE: We did'nt convert mask to RGB
        # as each color corresponds to a different instance with 0 being background
        mask = Image.open(mask_path)
        mask = np.array(mask)
        # Instances are encoded as different colors
        obj_ids = np.unique(mask)
        obj_ids = obj_ids[1:] # 1st ID is background so remove it.
        
        # Split the color-encoded mask into a set of binary mask
        num_objs = len(obj_ids)
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])

        # convert everything into a torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # there is only one class
        labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

### Define Model
We use `Mask R-CNN` which is based on top of `Faster R-CNN`. This model predicts both bounding boxes and class scores for potential objects in the image
<img src="https://pytorch.org/tutorials/_static/img/tv_tutorial/tv_image03.png"/>

Mask R-CNN adds an extra branch into `Faster R-CNN`, which also predicts segmentation masks for each instance.
<img src="https://pytorch.org/tutorials/_static/img/tv_tutorial/tv_image04.png"/>

There are two common situations where one might want to modify one of the available models in torchvision modelzoo. The first is when we want to start from a pre-trained model, and just finetune the last layer. The other is when we want to replace the backbone of the model with a different one (for faster predictions, for example).

### 1. FineTuning from a Pretrained Model
If we want to start from a model pre-trained on `COCO` and want to finetune it for your particular classes. Like this:

In [3]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# Load a model on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# Replace Classifier with a new one, that has
num_classes = 2 # 1 class + background
# Get no. of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# Replace the pretrained head with a new one.
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to C:\Users\RC/.cache\torch\hub\checkpoints\fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


  0%|          | 0.00/160M [00:00<?, ?B/s]

### 2. Modifying Model to add a different backbone

In [5]:
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

# Load a Pretrained model for classification and return features
backbone = torchvision.models.mobilenet_v2(pretrained=True).features
# Faster R-CNN needs to know no. of output channels in a backbone. For MobileNetV2 -> 1280
backbone.out_channels = 1280

# Let's make the RPN generate 5 x 3 anchors per spatial location, with 5 different sizes and 3
# different aspect ratios. We have Tuple[Tuple[int]] becoz each feature map could potentially
# have different sizes and aspect ratios
anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                   aspect_ratios=((0.5, 1.0, 2.0),))

# Let's define what are the feature maps that will be used to perform the region of Intrest Cropping (ROI) 
# as well as rescaling. If your backbone returns a Tensor, featmap_names is expected to
# be [0]. Generally, the backbone should return an OrderedDict[Tensor], and in featmap_names you can 
# choose which feature map to use.
roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'], output_size=7, sampling_ratio=2)

# Put all together inside a single Faster R-CNN model
model = FasterRCNN(backbone, num_classes=2, rpn_anchor_generator=anchor_generator, box_roi_pool=roi_pooler)

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to C:\Users\RC/.cache\torch\hub\checkpoints\mobilenet_v2-b0353104.pth


  0%|          | 0.00/13.6M [00:00<?, ?B/s]

### Instance Segmentation Model for PennFudan Dataset

If we want to finetune a pretrained model, given that our dataset is small, so we'll use approach `1`.

We compute the instance segmentation masks, so we will use `Mask R-CNN`

In [6]:
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

def get_model_instance_segmentation(num_classes):
    # Load an instance segmentation model pretrained on COCO.
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
    
    # Get no. of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # Replace the Pre-Trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    
    # Now get the no. of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask, hidden_layer, num_classes)
    
    return model

### Helper Functions

In [9]:
import transforms as T

def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

### Testing `forward()` method

Before iterating over the dataset, it's good to see what the model expects during training and inference time on sample data.

In [11]:
import utils

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
dataset = PennFudanDataset('PennFudanPed', get_transform(train=True))
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle=True, num_workers=4, collate_fn=utils.collate_fn)

# Training
images, targets = next(iter(data_loader))
images = list(image for image in images)
targets = [{k: v for k, v in t.items()} for t in targets]
output = model(images, targets) # Return Losses and detections

# Inference
model.eval()
x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
predictions = model(x) # Return Predictions

RuntimeError: DataLoader worker (pid(s) 3508, 25128, 30604, 36956) exited unexpectedly